In [34]:
import os
import csv
from collections import defaultdict
import pandas as pd

In [52]:
dict={}
protein_presence = defaultdict(lambda: defaultdict(int))
file2='GCA_019244415.1_ASM1924441v1_protein.gpff'
for file_name in os.listdir('.'):
    # 如果文件以.gpff结尾
    if file_name.endswith('.gpff'):
        with open(file_name, 'r') as file:
            bacteria_name=file_name.split('_A')[0]
            dict[bacteria_name]=[]
            # 读取文件内容
            file_contents = file.read()
            # 将内容按//分割
            proteins = file_contents.split('\n//\n')
            # 遍历每个蛋白质
            # print(proteins[1])
            for protein in proteins:
                
                # 将蛋白质按行分割，获取第二行
                lines = protein.split('\n')
                if len(lines) >= 2:
                    definition_line = lines[1]
                    if definition_line.startswith('DEFINITION'):
                        # 提取蛋白质名称
                        name = definition_line.split('DEFINITION')[1].split('[')[0].strip()
                        if 'MAG: ' in name:
                            name=name.split('MAG: ')[1]
                        # 添加到字典中，并标记为1
                        dict[bacteria_name].append(name)
                        protein_presence[file_name][name] = 1
                


In [65]:
my_dict = dict

with open('protein_dict.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(my_dict.keys())
    for row in zip(*my_dict.values()):
        writer.writerow(row)

In [58]:
# Create a set of unique (row, column) pairs
unique_pairs = set((row, column) for row, columns in my_dict.items() for column in columns)
# Convert the set to a list of rows
rows = [[row, column] for row, column in unique_pairs]
# Convert the list to a DataFrame
df = pd.DataFrame(rows, columns=['row', 'column'])
# Set the value to 1
df['value'] = 1
# Pivot the table
df = df.pivot(index='row', columns='column', values='value').fillna(0)
print(df)

column                                   \
row                                       
GCA_000019665.1                     1.0   
GCA_000020225.1                     0.0   
GCA_000025905.1                     0.0   
GCA_000155695.1                     0.0   
GCA_000172555.1                     0.0   
...                                 ...   
GCA_902143385.2_3B_2_protein.gpff   1.0   
GCA_902726595.1_LCC10_protein.gpff  1.0   
GCA_902805675.1                     0.0   
GCA_903064525.1_mthmo_protein.gpff  1.0   
GCA_905250085.1_MPQ17_protein.gpff  0.0   

column                              'Cold-shock' DNA-binding domain, putative  \
row                                                                             
GCA_000019665.1                                                           0.0   
GCA_000020225.1                                                           0.0   
GCA_000025905.1                                                           0.0   
GCA_000155695.1                    

In [62]:
# Compute the sum of each column
col_sums = df.sum(axis=0)

# Select the columns with sum greater than 10
selected_cols = col_sums[col_sums > 10].index.tolist()

# Filter the data frame to keep only selected columns
df = df[selected_cols]
